## Spark DataFrame Basics

In [40]:
import os

In [41]:
from pyspark.sql import SparkSession

In [42]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [43]:
data_file = os.path.join(os.curdir, 'data', 'people.json')

In [44]:
data = spark.read.json(data_file)

In [45]:
data.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [46]:
data.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [47]:
data.describe()

DataFrame[summary: string, age: string, name: string]

In [48]:
data.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [49]:
from pyspark.sql.types import (
    StructField, StringType,
    IntegerType, StructType
)

In [50]:
data_shema = [StructField('age', IntegerType(), True),
             StructField('name', StringType(), True)]

In [51]:
final_structure = StructType(fields=data_shema)

In [52]:
df = spark.read.json(data_file, schema=final_structure)

In [53]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [54]:
age_col = df.select('age') # the way to get age column from the df

In [55]:
age_col

DataFrame[age: int]

In [56]:
age_col.show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [57]:
df.select(['age', 'name'])

DataFrame[age: int, name: string]

In [58]:
# Adding new column
df.withColumn('newage', df['age']*2).show()

+----+-------+------+
| age|   name|newage|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    60|
|  19| Justin|    38|
+----+-------+------+



In [60]:
# Renaming column
df.withColumnRenamed('age', 'new_age')

DataFrame[new_age: int, name: string]

In [61]:
# create a view for Sql operations
df.createOrReplaceTempView('people')

In [62]:
result = spark.sql("SELECT * from people limit 1")

In [63]:
result.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
+----+-------+



In [64]:
apple_stock_file = os.path.join(os.curdir, 'data', 'appl_stock.csv')

In [65]:
apple_stock_df = spark.read.csv(apple_stock_file,
                               inferSchema=True,
                               header=True)

In [68]:
apple_stock_df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [71]:
apple_stock_df.head(3)

[Row(Date='2010-01-04', Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039),
 Row(Date='2010-01-05', Open=214.599998, High=215.589994, Low=213.249994, Close=214.379993, Volume=150476200, Adj Close=27.774976000000002),
 Row(Date='2010-01-06', Open=214.379993, High=215.23, Low=210.750004, Close=210.969995, Volume=138040000, Adj Close=27.333178000000004)]

In [84]:
# Applying filter

# apple_stock_df.filter("Close < 500").select(
#     ['Open', 'Close']).show()

In [85]:
# Another way of applying filter

apple_stock_df.filter(apple_stock_df['Close'] \
                      < 500).select('Volume').show()

+---------+
|   Volume|
+---------+
|123432400|
|150476200|
|138040000|
|119282800|
|111902700|
|115557400|
|148614900|
|151473000|
|108223500|
|148516900|
|182501900|
|153038200|
|152038600|
|220441900|
|266424900|
|466777500|
|430642100|
|293375600|
|311488100|
|187469100|
+---------+
only showing top 20 rows



In [89]:
apple_stock_df.filter((apple_stock_df['Close'] < 200) &\
                      (apple_stock_df['Open'] >200)).show()

# apple_stock_df.filter((apple_stock_df['Close'] < 200) &\
#                       ~(apple_stock_df['Open'] >200)).show()

+----------+------------------+----------+----------+----------+---------+------------------+
|      Date|              Open|      High|       Low|     Close|   Volume|         Adj Close|
+----------+------------------+----------+----------+----------+---------+------------------+
|2010-01-22|206.78000600000001|207.499996|    197.16|    197.75|220441900|         25.620401|
|2010-01-28|        204.930004|205.500004|198.699995|199.289995|293375600|25.819922000000002|
|2010-01-29|        201.079996|202.199995|190.250002|192.060003|311488100|         24.883208|
+----------+------------------+----------+----------+----------+---------+------------------+



In [91]:
# Collecting the result (being able to assign the result back to var)
result = apple_stock_df.filter(apple_stock_df['Low'] == 197.16).collect()

In [94]:
row = result[0]
row

Row(Date='2010-01-22', Open=206.78000600000001, High=207.499996, Low=197.16, Close=197.75, Volume=220441900, Adj Close=25.620401)

In [95]:
row.asDict()

{'Date': '2010-01-22',
 'Open': 206.78000600000001,
 'High': 207.499996,
 'Low': 197.16,
 'Close': 197.75,
 'Volume': 220441900,
 'Adj Close': 25.620401}

In [96]:
row.asDict().get('Open')

206.78000600000001

### Group by and aggregate

In [98]:
sales_data_file = os.path.join(os.curdir, 'data', 'sales_info.csv')
sales_data = spark.read.csv(sales_data_file,
                           inferSchema=True,
                           header=True)

In [100]:
sales_data.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [102]:
sales_data.groupBy("Company")

In [103]:
sales_data.groupBy("Company").mean().show()

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [104]:
sales_data.groupBy("Company").max().show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



In [108]:
sales_data.agg({'Sales': 'max'}).show()

# Finding maximum sales by company
grouped_data = sales_data.groupBy('Company')


+----------+
|max(Sales)|
+----------+
|     870.0|
+----------+



### Import spark functions

In [114]:
from pyspark.sql.functions import countDistinct,\
avg, stddev, format_number

In [115]:
sales_data.select(countDistinct('Sales')).show()

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



In [116]:
sales_data.select(avg('Sales').alias('Average Sales')).show()

+-----------------+
|    Average Sales|
+-----------------+
|360.5833333333333|
+-----------------+



In [123]:
std_sales = sales_data.select(stddev('Sales'))

std_sales.select(format_number('stddev_samp(Sales)',2).alias('std.')).show()

+------+
|  std.|
+------+
|250.09|
+------+



In [128]:
sales_data.orderBy('Sales').show() # ascending order

sales_data.orderBy(sales_data['Sales'].desc()).show() # descending order

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+

